In [4]:
import pandas as pd
import geojson

In [ ]:
#be careful running cell, uses a lot of memory
geojsonlocation = "./Data/Geojson/ConDistricts"
geoyears = [2012,2016,2020]
for gyear in geoyears:
    geofile = geojsonlocation+str(gyear)+".geojson"
    with open(geofile, 'w') as f:
        geojson_data = geojson.load(f)
        print(geojson_data)

In [3]:
elecbydist = pd.read_csv("./Data/election_results_by_district.csv")
elecbydist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855 entries, 0 to 854
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   County_dem_votes  855 non-null    float64
 1   County_rep_votes  855 non-null    float64
dtypes: float64(2)
memory usage: 13.5 KB


In [6]:
year = 2012

geofile = "./Data/Geojson/ConDistricts2020.geojson"
with open(geofile,'r') as f:
    geodata = geojson.load(f)
    elecbydist = pd.read_csv("./Data/election_results_by_district.csv")
    for feat in geodata['features']:
        proptoadd = {}
        samedist = elecbydist.loc[(elecbydist["Year"] == 2020)&(elecbydist["state_fips"]==int(feat.properties['STATEFP']))&(elecbydist["Congressional_District"]==feat.properties['NAMELSAD'])]
        if samedist.empty:
            proptoadd["WINNER"] = "Unknown"
            proptoadd["DEMVOTES"] = 0
            proptoadd["REPVOTES"] = 0
        else:
            if (samedist["District_dem_votes"].item() > samedist["District_rep_votes"].item()):
                proptoadd["WINNER"] = "Democrat"
                proptoadd["DEMVOTES"] = samedist["District_dem_votes"].item()
                proptoadd["REPVOTES"] = samedist["District_rep_votes"].item()
            else:
                proptoadd["WINNER"] = "Republican"
                proptoadd["DEMVOTES"] = samedist["District_dem_votes"].item()
                proptoadd["REPVOTES"] = samedist["District_rep_votes"].item()           
        feat.properties.update(proptoadd)
    with open("./Data/Geojson/RevConDistricts2020.geojson",'w') as out:
        geojson.dump(geodata,out)
    #for feature in geodata: geodata[60].properties['NAMELSAD']
        